This is the section for the group project proposal for group 9!

Here is the link to the data set we will be using, Online News Populartiy from the UCI Machine Learning Resprotory.
Subsiquent sources can be found in the link on the UCI web page.
Link: https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

Quesiton: Are we able to use previous articles to predict the day of the week an article was published?

In the cell below We will be importing all of the packages to help with our project!

In [1]:
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In the cell below, we will be reading our data into R.

In [2]:
getwd()

[1] "/home/jovyan/DCSI-100-Group-9-Project/project_proposal"

In [3]:
popularity<-read_csv("/home/jovyan/DCSI-100-Group-9-Project/data/OnlineNewsPopularity.csv")
head(popularity)

Rows: 39644 Columns: 61
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): url
dbl (60): timedelta, n_tokens_title, n_tokens_content, n_unique_tokens, n_no...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,⋯,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,12,219,0.6635945,1,0.8153846,4,2,1,⋯,0.10000000,0.7,-0.3500000,-0.600,-0.2000000,0.5000000,-0.1875000,0.00000000,0.1875000,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,9,255,0.6047431,1,0.7919463,3,1,1,⋯,0.03333333,0.7,-0.1187500,-0.125,-0.1000000,0.0000000,0.0000000,0.50000000,0.0000000,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,9,211,0.5751295,1,0.6638655,3,1,1,⋯,0.10000000,1.0,-0.4666667,-0.800,-0.1333333,0.0000000,0.0000000,0.50000000,0.0000000,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,9,531,0.5037879,1,0.6656347,9,0,1,⋯,0.13636364,0.8,-0.3696970,-0.600,-0.1666667,0.0000000,0.0000000,0.50000000,0.0000000,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.4156456,1,0.5408895,19,19,20,⋯,0.03333333,1.0,-0.2201923,-0.500,-0.0500000,0.4545455,0.1363636,0.04545455,0.1363636,505
http://mashable.com/2013/01/07/beewi-smart-toys/,731,10,370,0.5598886,1,0.6981982,2,2,0,⋯,0.13636364,0.6,-0.1950000,-0.400,-0.1000000,0.6428571,0.2142857,0.14285714,0.2142857,855


The next lines of Code will be used for wragling the data. I will be combining columns as well as removing extraneous columns I dont feel we need for this analysis. 

In [8]:

popularity_tidy <- popularity |> 
#removing columns that are specifically not litsted as predictors as well as columns that self reference articles.
select(-url, -timedelta , - num_self_hrefs, -self_reference_min_shares:-self_reference_avg_sharess,
      -is_weekend, -kw_min_min:-kw_avg_avg) |>

#The next line of code removes null values for day of the week.
#Since this data set makes each day boolean, we must check that each article 
#has been published on a day (i.e. one of the monday - sunday columns has a value 1 for each article)
#same is true for article type removing nulls.
filter(weekday_is_monday == 1| weekday_is_tuesday == 1| weekday_is_wednesday == 1| weekday_is_thursday == 1|
       weekday_is_friday == 1| weekday_is_saturday == 1| weekday_is_sunday == 1) |>

filter(data_channel_is_lifestyle == 1 | data_channel_is_entertainment == 1 | data_channel_is_bus == 1|
       data_channel_is_socmed == 1| data_channel_is_tech ==1 | data_channel_is_world == 1) |>

#Converts the days of the week and article type to factors from dbl
mutate(monday = as_factor(weekday_is_monday), tuesday = as_factor(weekday_is_tuesday), 
      wednesday = as_factor(weekday_is_wednesday) , thursday = as_factor(weekday_is_thursday),
      friday = as_factor(weekday_is_friday), saturday = as_factor(weekday_is_saturday),
       sunday = as_factor(weekday_is_sunday)) |>

mutate(lifestyle = as_factor(data_channel_is_lifestyle), 
       entertainment = as_factor(data_channel_is_entertainment), 
       bus = as_factor(data_channel_is_bus), 
       socmed = as_factor(data_channel_is_socmed), 
       tech = as_factor(data_channel_is_tech), 
       world = as_factor(data_channel_is_world)) |>

#combines the days of week and type data into their respective columns.
pivot_longer(cols = lifestyle:world, 
             names_to = "article_type",
             values_to = "yes") |>
filter(yes == "1") |>

pivot_longer(cols = monday:sunday, 
             names_to = "pub_day",
             values_to = "yes_2") |>
filter(yes_2 == "1") |>

select(-yes,-yes_2)


popularity_tidy

n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,⋯,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,article_type,pub_day
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
12,219,0.6635945,1,0.8153846,4,1,0,4.680365,5,⋯,-0.3500000,-0.6000000,-0.200000000,0.5000000,-0.1875000,0.00000000,0.1875000,593,entertainment,monday
9,255,0.6047431,1,0.7919463,3,1,0,4.913725,4,⋯,-0.1187500,-0.1250000,-0.100000000,0.0000000,0.0000000,0.50000000,0.0000000,711,bus,monday
9,211,0.5751295,1,0.6638655,3,1,0,4.393365,6,⋯,-0.4666667,-0.8000000,-0.133333333,0.0000000,0.0000000,0.50000000,0.0000000,1500,bus,monday
9,531,0.5037879,1,0.6656347,9,1,0,4.404896,7,⋯,-0.3696970,-0.6000000,-0.166666667,0.0000000,0.0000000,0.50000000,0.0000000,1200,entertainment,monday
13,1072,0.4156456,1,0.5408895,19,20,0,4.682836,7,⋯,-0.2201923,-0.5000000,-0.050000000,0.4545455,0.1363636,0.04545455,0.1363636,505,tech,monday
10,370,0.5598886,1,0.6981982,2,0,0,4.359459,9,⋯,-0.1950000,-0.4000000,-0.100000000,0.6428571,0.2142857,0.14285714,0.2142857,855,tech,monday
8,960,0.4181626,1,0.5498339,21,20,0,4.654167,10,⋯,-0.2244792,-0.5000000,-0.050000000,0.0000000,0.0000000,0.50000000,0.0000000,556,lifestyle,monday
12,989,0.4335736,1,0.5721078,20,20,0,4.617796,9,⋯,-0.2427778,-0.5000000,-0.050000000,1.0000000,0.5000000,0.50000000,0.5000000,891,tech,monday
11,97,0.6701031,1,0.8367347,2,0,0,4.855670,7,⋯,-0.1250000,-0.1250000,-0.125000000,0.1250000,0.0000000,0.37500000,0.0000000,3600,tech,monday
